In [14]:
# importing libraries
import os
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
main_folder = "data"

station_folders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]
station_folders

['Cascata', 'Piratininga']

In [16]:
column_rename_dict = {
    "PM 2,5": "PM2.5",
    "Partículas Respiráveis (<2,5µm)(µg/m3)": "PM2.5",
    "Partículas Inaláveis (<10µm)(µg/m3)": "PM10",
    "Temperatura (ºC)": "Temperatura",
    "Tº": "Temperatura",
    "Radiação Solar (W/m2)": "Radiação Solar",
    "Monóxido de Carbono (ppm)": "Monóxido de Carbono",
    "CO": "Monóxido de Carbono",
    "Velocidade Escalar do Vento (m/s)": "Velocidade do Vento",
    "Direção Escalar do Vento (grau)": "Direção do Vento",
    "Umidade relativa (%)": "Umidade Relativa",
    "Precipitação Pluviométrica (mm)": "Precipitação",
    "Pressão Atmosférica (mmHg)": "Pressão Atmosférica",
    "Dióxido de enxofre (ppb)": "Dióxido de Enxofre",
    "Ozônio (ppb)": "Ozônio",
    "Monóxido de Nitrogênio (ppb)": "Monóxido de Nitrogênio",
    "Óxidos de Nitrogênio (ppb)": "Óxidos de Nitrogênio",
    "Dióxido de Nitrogênio (ppb)": "Dióxido de Nitrogênio",
    "HCT (ppm)": "HCT",
    "CH4 (ppm)": "Metano",
    "HCNM (ppm)": "HCNM",
    "Monóxido de Carbono Média Móvel (ppm)": "Monóxido de Carbono Média Móvel",
    "Data": "Data",
    "Hora": "Hora",
    "Média Horária": "Média Horária",
    "Seno da direção do vento": "Seno da Direção do Vento",
    "Cosseno da direção do vento": "Cosseno da Direção do Vento",
    "Seno ponderado": "Seno Ponderado",
    "Cosseno ponderado": "Cosseno Ponderado",
    "PM10": "PM10",
    "NO2": "Dióxido de Nitrogênio",
    "O3": "Ozônio",
    "SO2": "Dióxido de Enxofre",
    "DV": "Direção do Vento",
    "PP": "Pressão Atmosférica",
    "PA": "Precipitação",
    "VV": "Velocidade do Vento",
    "RS": "Radiação Solar",
    "UR": "Umidade Relativa"
}
import pandas as pd
import os

# Supondo que column_rename_dict já esteja definido

for station_folder in station_folders:
    dfs_until_2014 = [] # Lista para acumular DataFrames
    dfs_from_2015 = [] # Lista para acumular DataFrames

    files = [f for f in os.listdir(os.path.join(main_folder, station_folder)) if f.endswith(".xlsx")]
    for file in files:
        year = int(file.split("_")[1].split(".")[0])
        file_path = os.path.join(main_folder, station_folder, file)
        df = pd.read_excel(file_path, skiprows=lambda x: x in [0, 2] if year >= 2015 else None)

        if year < 2015:
            df['Data'] = pd.to_datetime(df['Data']).dt.date
            dfs_until_2014.append(df)
        else:
            if 'Data e Hora' not in df.columns:
                df.rename(columns={df.columns[0]: 'Data e Hora'}, inplace=True)
            df['Data e Hora'] = df['Data e Hora'].astype(str)
            df[['Data', 'Hora']] = df['Data e Hora'].str.split(' ', n=1, expand=True)
            df = df.drop(columns=['Data e Hora'])
            cols = df.columns.tolist()
            cols = cols[-2:] + cols[:-2]
            df = df[cols]
            dfs_from_2015.append(df)

    # Concatenando os DataFrames acumulados
    combined_data = pd.concat(dfs_until_2014 + dfs_from_2015, ignore_index=True)

    # Aplicando renomeação de colunas uma única vez
    combined_data.rename(columns=column_rename_dict, inplace=True)

    # Filtrando e convertendo a coluna 'Data'
    combined_data = combined_data[combined_data['Data'] != '00/01/1900']
    combined_data['Data'] = pd.to_datetime(combined_data['Data'], format='%d-%m-%Y', errors='coerce')
    combined_data = combined_data.dropna(subset=['Data'])
    combined_data['Data'] = combined_data['Data'].dt.strftime('%d-%m-%Y')

    # Salvando o DataFrame combinado
    combined_data.to_excel(f"{station_folder}_combined.xlsx", index=False)


In [18]:
import pandas as pd
import os
import matplotlib.pyplot as plt

main_folder = "data"
station_folders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]

for station_folder in reversed(station_folders):
    combined_data = pd.read_excel(f"{station_folder}_combined.xlsx")
    # Especificando o formato correto da data
    combined_data['Data'] = pd.to_datetime(combined_data['Data'], format="%d-%m-%Y")
    combined_data = combined_data.set_index('Data')
    combined_data['PM2.5'].plot()
    plt.title(f"{station_folder} PM2.5")
    plt.show()


TypeError: no numeric data to plot